In [3]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data = pd.read_csv('/content/mobile_price_classification.csv')

In [5]:
data.head()

,battery_power,bluetooth,clock_speed,dual_sim,front_cam,4G,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   battery_power   2000 non-null   int64  
 1   bluetooth       2000 non-null   int64  
 2   clock_speed     2000 non-null   float64
 3   dual_sim        2000 non-null   int64  
 4   front_cam       2000 non-null   int64  
 5   4G              2000 non-null   int64  
 6   int_memory      2000 non-null   int64  
 7   m_dep           2000 non-null   float64
 8   mobile_wt       2000 non-null   int64  
 9   n_cores         2000 non-null   int64  
 10  primary_camera  2000 non-null   int64  
 11  px_height       2000 non-null   int64  
 12  px_width        2000 non-null   int64  
 13  ram             2000 non-null   int64  
 14  sc_h            2000 non-null   int64  
 15  sc_w            2000 non-null   int64  
 16  talk_time       2000 non-null   int64  
 17  three_g         2000 non-null   i

In [7]:
data.duplicated().sum()

0

In [8]:
data.isna().sum()

,0
battery_power,0
bluetooth,0
clock_speed,0
dual_sim,0
front_cam,0
4G,0
int_memory,0
m_dep,0
mobile_wt,0
n_cores,0


In [9]:
pip install tensorflow keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 10.1 MB/s eta 0:00:00


In [10]:
x = data.drop('price_range',axis=1)
y = data['price_range']

In [11]:
#splitting of dataset into test and train
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.33,random_state=0)


In [12]:
y.value_counts()

,count
price_range,
1,500
2,500
3,500
0,500


In [13]:
# Feature Scaling because yes we don't want one independent variable dominating the other and it makes computations easy
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [14]:
#one hot encode the target variable
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import keras_tuner as kt

In [16]:
x_train.shape

(1340, 20)

In [17]:
# Initialize the Model: Create a sequential model where layers can be added one after another.
classifier = Sequential()

In [18]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu', input_dim = 20))
##  first hidden layer with units=10, meaning this layer will contain 10 neurons.
#   These neurons will take the 20 input features and learn to transform them through their activation functions (in this case, ReLU).

# Adding the second hidden layer
classifier.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer of 4 neuron
classifier.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


compile the model

In [19]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

train the model


In [20]:
# Fitting the ANN to the Training set
classifier.fit(x_train, y_train, batch_size = 10, epochs = 50,verbose = 0)

# Instead of processing the entire dataset at once (which can be computationally expensive), the data is divided into smaller batches.
# In this case, the model processes 10 samples at a time before updating the weights.

#Epcohs: indicates how many times the learning algorithm will work through the entire training dataset.
# Here, the model will go through the entire dataset 100 times.
# More epochs can lead to better learning but also increases the risk of overfitting if not monitored properly

#verbose : Controls the amount of output printed during training.
# verbose=0: No output is printed during training.
# verbose=1: A progress bar is displayed (default).
# verbose=2: Displays one line per epoch.

In [21]:
#predicting the test results
y_pred = classifier.predict(x_test)
y_pred = (y_pred > 0.5)

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


Evaluation

In [22]:
score, acc = classifier.evaluate(x_test, y_test,batch_size=10)
print('Test score:', score)
print('Test accuracy:', acc)

66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9649 - loss: 0.0856
Test score: 0.08578502386808395
Test accuracy: 0.9651514887809753


Model summary

In [23]:
# Summarize the model
classifier.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 10)                  │             210 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             110 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 4)                   │              44 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,094 (4.28 KB)

 Trainable params: 364 (1.42 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 730 (2.86 KB)

In [24]:
def build_model(hp):
    model = Sequential()

    # Define the number of layers as a hyperparameter
    num_layers = hp.Int('num_layers', min_value=2, max_value=4, step=1)

    # Add layers based on `num_layers` hyperparameter
    for i in range(num_layers):
        units = hp.Int(f'units_{i+1}', min_value=16, max_value=128, step=16)
        model.add(Dense(units=units, activation='relu'))

    # Add output layer
    model.add(Dense(4, activation='softmax'))  # Assuming 4 classes

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [25]:
tuner = kt.RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials = 5
)

In [26]:
tuner.search(
    x_train,y_train,
    epochs =10,
    validation_split = 0.2,
    verbose = 1
)

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.8656716346740723

Best val_accuracy So Far: 0.9328358173370361
Total elapsed time: 00h 00m 22s


In [27]:
# Retrieve the best hyperparameters
best_hps = tuner.get_best_hyperparameters(1)[0]

# Print the units in the first layer and the number of layers
print('Units in 1st layer:', best_hps.get('units_1'))
print('Number of layers:', best_hps.get('num_layers'))

# Iterate through the layers and print the number of units for each layer
for i in range(best_hps.get('num_layers')):
    # Correct the f-string and variable naming for each layer's units
    print(f'Units in layer {i+1}: {best_hps.get(f"units_{i+1}")}')


Units in 1st layer: 96
Number of layers: 3
Units in layer 1: 96
Units in layer 2: 96
Units in layer 3: 112
